# 中文连续词袋模型 (CBOW) 实现

In [33]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
import jieba  # 用于中文分词

sentence = "自然语言处理是人工智能的一个重要分支。它涉及计算机对人类语言的理解和生成。"

# 1. 中文分词
seg_list = jieba.cut(sentence, cut_all=False)  # 使用精确模式分词
segmented_sentence = " ".join(seg_list)

print(f"分词结果: {segmented_sentence}")

# 2. 数据预处理
tokenizer = Tokenizer()
tokenizer.fit_on_texts([segmented_sentence])

word_index = tokenizer.word_index
vocabulary_size = len(word_index) + 1  # +1 for the padding token (if needed)

print(f"词汇表大小: {vocabulary_size}")
print(f"单词索引: {word_index}")

encoded_sentence = tokenizer.texts_to_sequences([segmented_sentence])[0]

def generate_cbow_pairs(corpus, window_size):
    context_length = 2 * window_size
    cbow_pairs = []
    for i in range(window_size, len(corpus) - window_size):
        context_words = [corpus[i - j] for j in range(window_size, 0, -1)]  # 前面的上下文词
        context_words += [corpus[i + j] for j in range(1, window_size + 1)]  # 后面的上下文词
        target_word = corpus[i]
        cbow_pairs.append((context_words, target_word))
    return cbow_pairs

window_size = 2
cbow_pairs = generate_cbow_pairs(encoded_sentence, window_size)

# 分离上下文词和目标词
context_words, target_words = zip(*cbow_pairs)

# 转换为 NumPy 数组
context_words = np.array(context_words)
target_words = np.array(target_words)

print(f"生成的 CBOW 词对数量: {len(cbow_pairs)}")
print(f"第一个词对: context={context_words[0]}, target={target_words[0]}")

# 4. 构建 CBOW 模型
embedding_dim = 50  # 词嵌入维度
context_length = 2 * window_size # 上下文长度

cbow_input = keras.Input(shape=(context_length,), dtype='int32') # 输入是上下文词序列
embedding = layers.Embedding(vocabulary_size, embedding_dim)(cbow_input)
average = layers.GlobalAveragePooling1D()(embedding) # 对词嵌入求平均
output = layers.Dense(vocabulary_size, activation='softmax')(average) # 使用 softmax 进行多分类

cbow = keras.Model(inputs=cbow_input, outputs=output)

cbow.summary()

# 5. 编译模型
cbow.compile(optimizer='adam',  # Adam 优化器
              loss='categorical_crossentropy',  # 适用于多分类问题
              metrics=['accuracy'])

# 将目标词转换为 one-hot 编码
target_words_one_hot = tf.keras.utils.to_categorical(target_words, num_classes=vocabulary_size)

print(f"context_words shape: {context_words.shape}")
print(f"target_words_one_hot shape: {target_words_one_hot.shape}")

# 6. 训练模型
epochs = 100

cbow.fit(context_words, target_words_one_hot, epochs=epochs, batch_size=256, verbose=1)  # 开启 verbose，查看训练过程

print("训练完成!")

# 7. 获取词嵌入
embedding_layer = cbow.layers[1]  # Embedding 层现在是第二层
embeddings = embedding_layer.get_weights()[0]

print(f"词嵌入矩阵的形状: {embeddings.shape}")  # (vocabulary_size, embedding_dim)

# 获取 "计算机" 的词嵌入
word = "计算机"
word_id = word_index[word]
embedding_vector = embeddings[word_id]

print(f"'{word}' 的词嵌入: {embedding_vector}")


分词结果: 自然语言 处理 是 人工智能 的 一个 重要 分支 。 它 涉及 计算机 对 人类 语言 的 理解 和 生成 。
词汇表大小: 19
单词索引: {'的': 1, '。': 2, '自然语言': 3, '处理': 4, '是': 5, '人工智能': 6, '一个': 7, '重要': 8, '分支': 9, '它': 10, '涉及': 11, '计算机': 12, '对': 13, '人类': 14, '语言': 15, '理解': 16, '和': 17, '生成': 18}
生成的 CBOW 词对数量: 16
第一个词对: context=[3 4 6 1], target=5


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_12 (Embedding)        │ (None, 4, 50)          │           950 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_9      │ (None, 50)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 19)             │           969 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,919 (7.50 KB)

 Trainable params: 1,919 (7.50 KB)

 Non-trainable params: 0 (0.00 B)

context_words shape: (16, 4)
target_words_one_hot shape: (16, 19)
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 2.9481   
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.1250 - loss: 2.9436
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.1875 - loss: 2.9390
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.1875 - loss: 2.9345
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.1875 - loss: 2.9300
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.2500 - loss: 2.9254
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.2500 - loss: 2.9209
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.2500 - loss: 2.9164
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.3750 - loss: 2.9118
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.3750 - loss: 2.9073
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5000 - loss: 2.9027
Epo